<a href="https://colab.research.google.com/github/326-YL/485nlp/blob/main/final2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import re
from tqdm import tqdm
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

%matplotlib inline

In [3]:
!pip install pandas
!pip install ktrain
!pip install tensorflow
!pip install tokenizers
!pip install protobuf
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 25.3/25.3 MB 56.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 43.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 55.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.8/468.8 kB 11.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Created wheel for ktrain: filename=ktrain-0.39.0-py3-none-any.whl size=25319737 sha256=1dd3aa3f4d2dc7ddc2ebe2c5242e7da0852f5bd09acbdca462b398485ba81e0a
  Stored in directory: /root/.cache/pip/wheels/98/fd/0a/ef6252223f3d2

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
path="/content/drive/MyDrive/dataset/AItAS_dataset-5.csv"
df=pd.read_csv(path)
df.drop(df.columns[[0]], axis=1, inplace=True)
print(df.head(10))
df['label'].value_counts()


                                                body            label
0  im on my phone sorry for errors. \r\n\r\ni jus...          Asshole
1  My eldest sister has had 18 months to plan to ...   Not the A-hole
2  Me and my family are on vacation for my sweet ...  No A-holes here
3  My boyfriend (28M) and myself (26F) live with ...   Not the A-hole
4  Throw away account for AITA. The fish are fine...   Everyone Sucks
5  So for a little context: my now exfriend( who ...   Not the A-hole
6  So, back around March of 2020, this lovely thi...  No A-holes here
7  I (22F) moved into my own apartment about thre...   Not the A-hole
8  I haven’t seen my dad in 10+ years and I have ...   Not the A-hole
9  My wife and I got married 6 months ago and my ...          Asshole


Not the A-hole     156848
Asshole             30989
No A-holes here      9852
Everyone Sucks       7910
Not enough info      4552
Name: label, dtype: int64

<strong>Data sampling </strong>

In [6]:
import ktrain
from sklearn.model_selection import train_test_split
from sklearn.utils import class_weight

In [14]:
classes =  list(set(df.label))


X_train, X_test, y_train, y_test = train_test_split(df.body, df.label, test_size=0.01, random_state=42)
X_train = X_train.astype(str)
X_test = X_test.astype(str)
# To fill NaN values with a placeholder
X_train.fillna('placeholder', inplace=True)
X_test.fillna('placeholder', inplace=True)

# Or to drop rows with NaN values
X_train.dropna(inplace=True)
X_test.dropna(inplace=True)


# Pre Processing
trn, val, preproc = ktrain.text.texts_from_array(x_train=X_train.values, y_train=y_train.values,
                                          x_test=X_test.values, y_test=y_test.values,
                                                                   class_names=classes,
                                                                   val_pct=0.1,
                                                                   max_features=30000,
                                                                   maxlen=350,
                                                                   preprocess_mode='distilbert',
                                                                   ngram_range=1)

# Model
model = ktrain.text.text_classifier('distilbert', train_data=trn, preproc=preproc)

preprocessing train...
language: en
train sequence lengths:
	mean : 369
	95percentile : 589
	99percentile : 702


/usr/local/lib/python3.10/dist-packages/ktrain/utils.py:744: UserWarning: class_names argument was ignored, as they were extracted from string labels in dataset
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Is Multi-Label? False
preprocessing test...
language: en
test sequence lengths:
	mean : 365
	95percentile : 590
	99percentile : 693


task: text classification
Is Multi-Label? False
maxlen is 350
done.


In [ ]:
# Handling class imbalance
from sklearn.utils import class_weight
class_weights = class_weight.compute_class_weight(class_weight='balanced', classes=np.unique(df.label),y=y_train)
from keras.callbacks import EarlyStopping
print(class_weights)

weights={}
for index, weight in enumerate(class_weights) :
  weights[index]=weight

print(weights)

# Applying the weights

learner =  ktrain.get_learner(model, train_data=trn, val_data=val,batch_size=6)

# you can find the learning rate
learner.lr_find(show_plot=True,max_epochs=5)

# you can use the learning rate obtained and run for 3 epochs
learner.autofit(2e-5, 3,class_weight = weights,callbacks=[EarlyStopping(monitor='val_loss', patience=3, min_delta=0.0001)])

[1.35558886 5.3141507  4.26898533 9.23021295 0.26798524]
{0: 1.3555888581202151, 1: 5.314150702426565, 2: 4.268985328819124, 3: 9.230212954747117, 4: 0.2679852385215336}
simulating training for different learning rates... this may take a few moments...
Epoch 1/5
